Actividad Guiada 3

Jordi Tudela

URL: https://github.com/bar0net/03MAIR---Algoritmos-de-Optimizacion/blob/master/AG3/JordiTudela-AG3.ipynb

In [1]:
import urllib.request
import tsplib95
import random
from math import e

file = "swiss42.tsp"
urllib.request.urlretrieve("http://elib.zib.de/pub/mp-testdata/tsp/tsplib/tsp/swiss42.tsp", file)

('swiss42.tsp', <http.client.HTTPMessage at 0x22ac16105f8>)

## Preparación de datos

In [2]:
problem = tsplib95.load_problem(file)
nodos   = list(problem.get_nodes())
aristas = list(problem.get_edges())

## Utilidades

In [3]:
def Factorial(n):
    if n == 0: 
        return 0
    
    output = 1
    for i in range(1, n+1):
        output *= i
    return output

def Distancia(a,b,problem):
    return problem.wfunc(a,b)

def DistanciaTotal(recorrido, problem):
    dst = 0
    for a,b in zip(recorrido[:-1], recorrido[1:]):
        dst += problem.wfunc(a,b)
    return dst

def Random(nodes):
    solution = [0]
    available = set(nodes) - set([0])
    
    for i in range(len(nodes) - 1):
        insert = random.choice(list(available))
        solution += [insert]
        available.remove(insert)
    return solution

def Prob(T,d):
    r = random.random()
    
    if r <= e**((-d)/(float(T))):
        return True
    else:
        return False

## Búsqueda Aleatoria

In [4]:
def BusquedaAleatoria(nodes, problem, iterations=1000):
    solution = Random(nodes)
    cost = DistanciaTotal(solution, problem)
    
    for _ in range(iterations-1):
        new_solution = Random(nodes)
        new_cost = DistanciaTotal(solution, problem)
        
        if new_cost < cost:
            solution, cost = new_solution, new_cost
            
    return cost, solution

print(BusquedaAleatoria(nodos, problem))
print(BusquedaAleatoria(nodos, problem))

(5370, [0, 8, 26, 34, 18, 2, 5, 16, 38, 21, 3, 12, 31, 40, 35, 13, 4, 25, 20, 27, 32, 29, 28, 14, 41, 24, 17, 10, 9, 36, 30, 6, 1, 22, 11, 37, 39, 33, 19, 23, 7, 15])
(5236, [0, 13, 21, 29, 5, 41, 33, 17, 9, 16, 32, 6, 28, 1, 8, 18, 15, 2, 34, 14, 40, 27, 4, 39, 38, 10, 25, 7, 22, 26, 36, 20, 3, 24, 19, 31, 23, 37, 11, 12, 35, 30])


## Búsqueda Local

In [5]:
def Vecina(solucion, problem):
    sol = []
    dst = float("Inf")
    m = len(solucion)
    
    for i in range(1,m-1):
        for j in range(i+1,m):
            vecina = list(solucion)
            vecina[i], vecina[j] = vecina[j], vecina[i]
            
            dst_vecina = DistanciaTotal(vecina, problem)
            if dst_vecina < dst:
                dst, sol = dst_vecina, vecina
    return dst, sol

print(Vecina(Random(nodos), problem))

(4372, [0, 29, 21, 23, 26, 27, 28, 38, 22, 4, 8, 33, 40, 3, 15, 35, 41, 10, 20, 14, 12, 19, 39, 32, 37, 11, 13, 6, 18, 17, 31, 36, 30, 2, 25, 5, 24, 9, 7, 34, 1, 16])


In [6]:
# En este caso generamos vecinos intentando invertir el orden 
# de sublistas internas (de tamaño creciente) 
def VecinaInvert(solucion, problem):
    sol = []
    dst = float("Inf")
    m = len(solucion)
    
    for i in range(1,m-1):
        for j in range(1,m-i):
            k = j+i
            vecina = solucion[:j] + list(reversed(solucion[j:k])) + solucion[k:]
            
            dst_vecina = DistanciaTotal(vecina, problem)
            if dst_vecina < dst:
                dst, sol = dst_vecina, vecina
    return dst, sol
print(VecinaInvert(Random(nodos), problem))

(4707, [0, 24, 4, 14, 17, 23, 11, 35, 36, 32, 15, 13, 29, 18, 10, 16, 8, 41, 38, 2, 20, 19, 26, 27, 34, 12, 31, 25, 21, 30, 5, 6, 37, 9, 33, 22, 28, 1, 7, 39, 3, 40])


In [7]:
# Igual que la solución anterior pero en lugar de realizar inversión de sublistas, realizamos
# n permutaciones aleatorias de las sublistas (por defecto 1 permutación)
def VecinaRandom(solucion, problem, iterations = 1):
    sol = []
    dst = float("Inf")
    m = len(solucion)
    
    for i in range(1,m-1):
        for j in range(1,m-i):
            for _ in range(iterations):
                k = j+i
                sublista = list(solucion[j:k])
                random.shuffle(sublista)
                vecina = solucion[:j] + sublista + solucion[k:]

                dst_vecina = DistanciaTotal(vecina, problem)
                if dst_vecina < dst:
                    dst, sol = dst_vecina, vecina
    return dst, sol
print(VecinaRandom(Random(nodos), problem, 10))

(3718, [0, 10, 31, 14, 4, 27, 5, 13, 29, 40, 9, 34, 26, 32, 6, 37, 11, 12, 1, 16, 19, 15, 36, 33, 20, 30, 8, 25, 23, 41, 21, 28, 39, 17, 22, 24, 38, 35, 2, 18, 7, 3])


In [8]:
def BusquedaLocal(problem, iteraciones=100, gen='sample'):
    nodos = list(problem.get_nodes())
    sol = Random(nodos)
    dst = float("Inf")
    
        
    for i in range(iteraciones):
        if gen == 'sample':
            new_dst, new_sol = Vecina(sol, problem)
        elif gen == 'invert':
            new_dst, new_sol = VecinaInvert(sol, problem)
        elif gen == 'random':
            new_dst, new_sol = VecinaRandom(sol, problem, 1)
            
        
        if new_dst < dst:
            sol, dst = new_sol, new_dst
    
    return dst, sol
print(BusquedaLocal(problem, iteraciones=100, gen='invert'))

(1358, [0, 20, 33, 34, 32, 27, 2, 28, 29, 30, 38, 22, 39, 21, 24, 40, 23, 9, 8, 41, 10, 25, 11, 12, 18, 4, 3, 1, 6, 26, 5, 13, 19, 14, 16, 15, 37, 17, 36, 35, 31, 7])


## Recocido Simulado

In [9]:
def RecocidoSimulado(problem, temperatura):
    sol = Random(list(problem.get_nodes()))
    dst = DistanciaTotal(sol, problem)
    
    ref = sol
    ref_dst = dst
    
    while temperatura > 0:
        new_dst, new_sol = Vecina(sol, problem)
        
        if new_dst < dst:
            sol, dst = new_sol, new_dst
            
        if new_dst < ref_dst and Prob(temperatura, abs(ref_dst-new_dst)):
            ref_sol, ref_dst = new_sol, new_dst            
        
        temperatura -= 1
        
    return dst, sol

print(RecocidoSimulado(problem, 1000))

(1484, [0, 2, 27, 34, 33, 20, 35, 36, 17, 31, 32, 30, 38, 22, 39, 29, 28, 6, 5, 13, 19, 14, 16, 15, 37, 7, 1, 3, 4, 26, 18, 12, 11, 25, 10, 8, 9, 21, 24, 40, 23, 41])


## Metric TSP

Implementar la computación de TSP usando la heurística "Metric". 

Metric TSP asegura encontrar un resultado menor o igual al doble del óptimo.

**Referencia:** 
- Coursera, *Estructuras de datos y algoritmos* (https://www.coursera.org/specializations/data-structures-algorithms)
  - Algorithms of Graphs: *Algoritmo de Prim*
  - Advanced Algorithms and Complexity: *Metric TSP*

In [10]:
# Metric TSP presume que el grafo completo cumple con la desigualdad triangular
nodes = problem.get_nodes()

def CheckTriangular(nodes):
    for i in nodes:
        for j in nodes:
            for k in nodes:
                if problem.wfunc(i,j) + problem.wfunc(j,k) < problem.wfunc(i,k):
                    print(i,j,k)
                    return False
    return True
print("Cumple desigualdad triangular:", CheckTriangular(nodes))

Cumple desigualdad triangular: True


### Minimum Spannig Trees

In [11]:
# Primero utilizamos el algoritmo de Prim para computar
# una solución al problema MST
def Prim(problem):
    nodos = list(problem.get_nodes())
    aristas = list(problem.get_edges())
    visited = set()
    
    # definimos las listas de costes y padres
    # Inicialmente, el coste de ir a cualquier nodo es infinito
    # y todos los nodos están desconectados (no tienen padre)
    parent = [None] * len(nodos)
    costes = [float('Inf')] * len(nodos)
    
    # Elegimos un nodo inicial al azar
    explore = [random.choice(nodos)]
    #explore = [0]
    costes[explore[0]] = 0
    
    while len(explore) > 0:
        # Exploramos las ramas priorizando las de menor coste
        explore = sorted(explore, key=lambda x : costes[x], reverse=True)
        current = explore.pop(0)
        visited.add(current)
        
        # Actualizamos el coste mínimo de moverse entre nodos si es necesario
        # (y actualizamos también el padre)
        for i, x in enumerate(nodos):
            if x in visited:
                continue
            
            dst = Distancia(current, x, problem)
            if dst < costes[i]:
                costes[i] = dst
                parent[i] = current
                
                explore.append(x)            
    return parent, costes, nodos

# Podemos mejorar la complejidad algoritmica usando 
# min-heap para almacenar los nodos a explorar!

In [12]:
def Cycle(graph, u, costs, prev = None):
    path = [u]
    
    candidates = list(graph[u])
    candidates = sorted(candidates, key=lambda x : costs[x], reverse=True)
    
    for v in candidates:
        if v == prev:
            continue
        
        path += Cycle(graph, v, costs, u) + [u]
        
    return path

In [13]:
def MetricTSP(problem):
    # Encontrar MST
    parent, costs, nodos = Prim(problem)
    
    # Duplicar todos las aristas del MST
    graph = {x:[] for x in nodos}
    for p, x in zip(parent, nodos):
        if p == None:
            continue
        graph[p].append(x)
        graph[x].append(p)
    
    ciclo = Cycle(graph, 0, costs)
    
    # Generar ruta: vamos siguiendo el ciclo en orden
    # y añadimos nodos que no estén ya en la lista
    visited = [False] * len(nodos)
    path = []
    
    while len(ciclo) > 0:
        current = ciclo.pop(0)
        if not visited[current]:
            visited[current] = True
            path.append(current)
            
    # DEBUG: Nos aseguramos de visitar todos los nodos
    visited = [False] * len(list(problem.get_nodes()))
    for p in path:
        visited[p] = True
        
    for i,p in enumerate(visited):
        if not p:
            print("ERROR")
            
    return path
    
DistanciaTotal(MetricTSP(problem), problem)

2619

## Algoritmo de Christofides

**referencia:** R9. Approximation Algorithms: Traveling Salesman Problem (MIT OpenCourseWare, Youtube) https://www.youtube.com/watch?v=zM5MW5NKZJg

In [14]:
def DFScount(graf, u, visited):
    count = 1
    visited[u] = True
    for v in graf[u]:
        if not visited[v]:
            count += DFScount(graf, v, visited)
    return count

def FleuryValid(graph, u, v):
    if len(graph[u]) == 1:
        return True
    
    visited = [False]*len(graph.keys())
    countA = DFScount(graph, u, visited)
    
    visited = [False]*len(graph.keys())
    graph[u].remove(v)
    graph[v].remove(u)
    countB = DFScount(graph, u, visited)
    graph[u].append(v)
    graph[v].append(u)
    
    return countA <= countB

def Fleury(graph, u):
    output = []
    candidates = list(graph[u])
    
    for v in candidates:
        if v in graph[u] and FleuryValid(graph, u,v):
            output.append(v)
            graph[u].remove(v)
            graph[v].remove(u)
            output += Fleury(graph, v)  
            
    return output

In [15]:
def Connect(graph, odds, problem):
    # Usamos un algoritmo de poda para tratar de encontrar la colección
    # de aristas que minimizan su peso conjunto
    maximum = max(problem.edge_weights)
    m = len(odds)
    
    # poblaremos explore con tuplas que contienen:
    #  - cota inferios
    #  - cota superior
    #  - un set con todos los nodos visitados en la construcción de esa tupla
    #  - una lista de parejas visitadas en la construcción de esa tupla
    
    explore = [(0, maximum * (m) / 2, set(), [] )]
    count = 0
    while len(explore) > 0:
        
        explore = sorted(explore, key=lambda x : x[1])  
        
        ci, cs, unique, pairs = explore.pop(0)
        if ci == cs:
            break
        
        # Encontrar el primer índice no usado
        for idx in range(m):
            if idx not in unique:
                unique.add(idx)
                break
                
        for i in range(m):
            if i in unique:
                continue
            
            unique.add(i)
            cost = ci + problem.wfunc(odds[idx],odds[i])
            explore.append( (cost, cost + maximum * (m - len(unique))/2, set(list(unique)), pairs + [(idx,i)] ) )
            unique.remove(i)
    
    # como construimos la tupla con indices, debemos
    # hacer la conversión de indice a nodo para completar
    # el grafo
    for i,j in pairs:
        graph[odds[i]].append(odds[j])
        graph[odds[j]].append(odds[i])  
    
    

def Christofides(problem):
    # Encontrar MST
    parent, costs, nodos = Prim(problem)
    
    # Duplicar todos las aristas del MST
    graph = {x:[] for x in nodos}
    for p, x in zip(parent, nodos):
        if p == None:
            continue
        graph[p].append(x)
        graph[x].append(p)
        
    # Encontrar vertices con un número impar
    # de aristas en el MST
    impar = []
    for key, value in graph.items():
        if len(value) % 2 == 1:
            impar.append(key)
    
    # Añadimos aristas
    Connect(graph, impar, problem)
    
    # Buscamos un ciclo euleriano para el grafo
    ciclo = [0] + Fleury(graph, 0)
    
    # Creamos un camino siguiendo el ciclo 
    # y obviando nodos ya visitados
    visited = [False] * len(nodos)
    path = []
    while len(ciclo) > 0:
        current = ciclo.pop(0)
        if not visited[current]:
            visited[current] = True
            path.append(current)
            
    visited = [False] * len(list(problem.get_nodes()))
    for p in path:
        visited[p] = True
        
    for i,p in enumerate(visited):
        if not p:
            print("ERROR")
            
    return path
    
DistanciaTotal(Christofides(problem), problem)

2196